In [ ]:
import pandas as pd

# 匯入原本的 CSV 檔案
input_path = r'C:\Users\User\Dropbox\textmining1\Data_Clean\modify_merged_file1.csv'  # 請將這裡替換為你的原始檔案路徑
data = pd.read_csv(input_path)

# 建立替換對應字典
replacement_map = {
    'b1': 'b2', 'b2': 'b3', 'b3': 'b4', 'b4': 'b5',
    'b5': 'b6', 'b6': 'b7', 'b7': 'b8', 'b8': 'b9',
    'b9': 'b10', 'b10': 'b1'
}

# 使用 replace 方法進行替換
data['pdf'] = data['pdf'].replace(replacement_map)

# 將修改後的資料輸出為新的 CSV 檔案
output_path = r'C:\Users\User\Dropbox\textmining1\Data_Clean\modify_merged_file2.csv'  # 請替換為你想儲存的檔案路徑
data.to_csv(output_path, index=False)

print(f"修改後的檔案已儲存為: {output_path}")


修改後的檔案已儲存為: C:\Users\User\Dropbox\textmining1\Data_Clean\modify_merged_file1.csv


In [ ]:
import pandas as pd
import re

# CSV檔案路徑
input_path = r'C:\Users\User\Dropbox\textmining1\Data_Clean\modify_merge.csv'
output_path = r'C:\Users\User\Dropbox\textmining1\Data_Clean\sorted_modify_merge.csv'

# 讀取CSV檔案
data = pd.read_csv(input_path)

# 定義排序邏輯
def custom_sort(value):
    # 使用正則表達式拆解字母和數字部分
    match = re.match(r'([a-zA-Z]+)(\d+)?', str(value))
    if match:
        letter_part = match.group(1)  # 提取字母部分
        number_part = int(match.group(2)) if match.group(2) else 0  # 提取數字部分
        return (letter_part, number_part)  # 返回字母和數字作為排序依據
    return (value, float('inf'))  # 若無匹配，放在最後

# 按照自定義邏輯排序 'pdf' 欄位
data = data.sort_values(by='pdf', key=lambda col: col.map(custom_sort))

# 將排序後的資料儲存為新的CSV檔案
data.to_csv(output_path, index=False)

print(f"排序後的檔案已儲存於: {output_path}")


In [2]:
import pandas as pd
import re

# CSV檔案路徑
input_path = r'C:\Users\User\Dropbox\textmining1\Data_Clean\modify_merge.csv'
output_path = r'C:\Users\User\Dropbox\textmining1\Data_Clean\sorted_modify_merge.csv'

# 讀取數據
data = pd.read_csv(input_path)

# 確保 PDF 欄位為字串類型
data['PDF'] = data['PDF'].astype(str)

# 定義 PDF 排序邏輯
def custom_sort(value):
    match = re.match(r'([a-zA-Z]+)(\d+)', value)
    if match:
        letter_part = match.group(1)  # 提取字母部分
        number_part = int(match.group(2))  # 提取數字部分
        return (letter_part, number_part)  # 返回字母和數字排序依據
    return (value, float('inf'))  # 無法匹配時放到最後

# 按 PDF 排序
data = data.sort_values(by='PDF', key=lambda col: col.map(custom_sort))

# 分組內按照 Case Index 排序
grouped_sorted_data = []
for _, group in data.groupby('PDF', sort=False):  # 按 PDF 分組
    group = group.sort_values(by='Case Index')  # 每組內按 Case Index 排序
    grouped_sorted_data.append(group)

# 合併排序後的分組結果
final_data = pd.concat(grouped_sorted_data).reset_index(drop=True)

# 儲存排序後的結果
final_data.to_csv(output_path, index=False)

print(f"分組並排序後的檔案已儲存於: {output_path}")

分組並排序後的檔案已儲存於: C:\Users\User\Dropbox\textmining1\Data_Clean\sorted_modify_merge.csv


In [32]:
import pandas as pd
import glob
import re
import os

# 指定存放所有 CSV 檔案的目錄和輸出路徑
input_folder = r'C:\Users\User\Dropbox\textmining1\Data_Clean\Opinion_Index'
output_path = r'C:\Users\User\Dropbox\textmining1\Data_Clean\Merged_Opinion_Index\Merged_Opinion_Index.csv'

# 定義自然排序函數
def custom_sort(value):
    match = re.match(r'([a-zA-Z]+)(\d+)?', str(value))
    if match:
        letter_part = match.group(1)
        number_part = int(match.group(2)) if match.group(2) else 0
        return (letter_part, number_part)
    return (value, float('inf'))

# 按自然順序讀取所有 CSV 檔案
all_files = glob.glob(f"{input_folder}/*.csv")
all_files_sorted = sorted(all_files, key=custom_sort)

# 檢查讀取到的檔案
print(f"發現的檔案數量：{len(all_files_sorted)}")
print("檔案清單：")
for file in all_files_sorted:
    print(f"- {os.path.basename(file)}")

dataframes = []
failed_files = []

for file in all_files_sorted:
    try:
        # 讀取檔案
        df = pd.read_csv(file, encoding='utf-8')
        # 清理欄位名稱
        df.columns = df.columns.str.strip()

        # 確保 'Page' 欄位存在
        if 'Page' not in df.columns:
            raise ValueError(f"'Page' 欄位不存在於檔案 {file}")
        
        # 打印檔案的基本結構
        print(f"檔案 {file} 的欄位名稱：{df.columns.tolist()}")
        print(df.head())

        # 添加來源檔案名稱欄位
        df['CSV'] = os.path.basename(file)
        dataframes.append(df)

    except Exception as e:
        print(f"檔案 {file} 讀取失敗，錯誤信息：{e}")
        failed_files.append(file)

# 如果有未成功讀取的檔案，打印清單
if failed_files:
    print(f"以下檔案未成功讀取，共 {len(failed_files)} 個：")
    for file in failed_files:
        print(f"- {os.path.basename(file)}")

# 合併所有有效的數據框
if dataframes:
    merged_data = pd.concat(dataframes, ignore_index=True)

    # 合併 Paragraph 並計算 Page 範圍
    result = (
        merged_data.groupby(['CSV', 'Case Index'])
        .agg({
            'Page': lambda x: f"{x.min()}-{x.max()}",  # 合併 Page 為範圍
            'Paragraph': lambda x: " ".join(x.dropna().astype(str))  # 合併 Paragraph
        })
        .reset_index()
    )

    # 排序數據
    result['CSV_Sort_Key'] = result['CSV'].apply(custom_sort)
    result = result.sort_values(by=['CSV_Sort_Key', 'Case Index']).drop(columns=['CSV_Sort_Key'])

    # 儲存結果，強制使用 UTF-8 編碼
    try:
        with open(output_path, "w", encoding="utf-8", newline='') as csv_file:
            result.to_csv(csv_file, index=False)
        print(f"合併並排序後的檔案已儲存於: {output_path}")
    except PermissionError:
        print(f"無法儲存檔案，請確保檔案未被其他程序使用：{output_path}")
else:
    print("未找到可用的數據，請檢查原始檔案。")


發現的檔案數量：113
檔案清單：
- a1.csv
- a10.csv
- a2.csv
- a3.csv
- a4.csv
- a5.csv
- a6.csv
- a7.csv
- a8.csv
- a9.csv
- b1.csv
- b10.csv
- b2.csv
- b3.csv
- b4.csv
- b5.csv
- b6.csv
- b7.csv
- b8.csv
- b9.csv
- c1.csv
- c10.csv
- c2.csv
- c3.csv
- c4.csv
- c5.csv
- c6.csv
- c7.csv
- c8.csv
- c9.csv
- d1.csv
- d10.csv
- d11.csv
- d12.csv
- d13.csv
- d14.csv
- d15.csv
- d16.csv
- d17.csv
- d18.csv
- d19.csv
- d2.csv
- d3.csv
- d4.csv
- d5.csv
- d6.csv
- d7.csv
- d8.csv
- d9.csv
- e1.csv
- e10.csv
- e2.csv
- e3.csv
- e4.csv
- e5.csv
- e6.csv
- e7.csv
- e8.csv
- e9.csv
- f1.csv
- f10.csv
- f2.csv
- f3.csv
- f4.csv
- f5.csv
- f6.csv
- f7.csv
- f8.csv
- f9.csv
- g1.csv
- g10.csv
- g2.csv
- g3.csv
- g4.csv
- g5.csv
- g6.csv
- g7.csv
- g8.csv
- g9.csv
- h1.csv
- h10.csv
- h2.csv
- h3.csv
- h4.csv
- h5.csv
- h6.csv
- h7.csv
- h8.csv
- h9.csv
- i1.csv
- i10.csv
- i2.csv
- i3.csv
- i4.csv
- i5.csv
- i6.csv
- i7.csv
- i8.csv
- i9.csv
- j1.csv
- j10.csv
- j2.csv
- j3.csv
- j4.csv
- j5.csv
- j6.csv
- j7.csv


In [35]:
print(result['CSV'].unique())


['a1.csv' 'a2.csv' 'a3.csv' 'a4.csv' 'a5.csv' 'a6.csv' 'a7.csv' 'a8.csv'
 'a9.csv' 'a10.csv' 'b1.csv' 'b2.csv' 'b3.csv' 'b4.csv' 'b5.csv' 'b6.csv'
 'b7.csv' 'b8.csv' 'b9.csv' 'b10.csv' 'c1.csv' 'c2.csv' 'c3.csv' 'c4.csv'
 'c5.csv' 'c6.csv' 'c7.csv' 'c8.csv' 'c9.csv' 'c10.csv' 'd1.csv' 'd2.csv'
 'd3.csv' 'd4.csv' 'd5.csv' 'd6.csv' 'd7.csv' 'd8.csv' 'd9.csv' 'd10.csv'
 'd11.csv' 'd12.csv' 'd13.csv' 'd14.csv' 'd15.csv' 'd16.csv' 'd17.csv'
 'd18.csv' 'd19.csv' 'e1.csv' 'e2.csv' 'e3.csv' 'e4.csv' 'e5.csv' 'e6.csv'
 'e7.csv' 'e8.csv' 'e9.csv' 'e10.csv' 'f1.csv' 'f2.csv' 'f3.csv' 'f4.csv'
 'f5.csv' 'f6.csv' 'f7.csv' 'f8.csv' 'f9.csv' 'f10.csv' 'g1.csv' 'g2.csv'
 'g3.csv' 'g4.csv' 'g5.csv' 'g6.csv' 'g7.csv' 'g8.csv' 'g9.csv' 'g10.csv'
 'h1.csv' 'h2.csv' 'h3.csv' 'h4.csv' 'h5.csv' 'h6.csv' 'h7.csv' 'h8.csv'
 'h9.csv' 'h10.csv' 'i1.csv' 'i2.csv' 'i3.csv' 'i4.csv' 'i5.csv' 'i6.csv'
 'i7.csv' 'i8.csv' 'i9.csv' 'i10.csv' 'j1.csv' 'j2.csv' 'j3.csv' 'j4.csv'
 'j5.csv' 'j6.csv' 'j7.csv' 'j8.csv' 'j9.c